In [ ]:
!pip install scipy

from scipy.optimize import linprog
import numpy as np

# Supply and demand
supply = [20, 30, 25]  # Supply from factories A, B, C
demand = [15, 25, 35]  # Demand at warehouses X, Y, Z

# Cost matrix (flattened for linprog)
cost = [4, 6, 8, 5, 3, 7, 6, 5, 3]  # Cost per unit for each route

# Coefficients matrix for constraints (supply and demand)
A_eq = []

# Supply constraints
for i in range(len(supply)):
    supply_constraint = [0] * len(cost)
    supply_constraint[i * len(demand):(i + 1) * len(demand)] = [1] * len(demand)
    A_eq.append(supply_constraint)

# Demand constraints
for j in range(len(demand)):
    demand_constraint = [0] * len(cost)
    for i in range(len(supply)):
        demand_constraint[i * len(demand) + j] = 1
    A_eq.append(demand_constraint)

# Right-hand side of constraints
b_eq = supply + demand  # Concatenating supply and demand constraints

# Bounds for each variable (shipment volumes must be non-negative)
x_bounds = [(0, None)] * len(cost)

# Solve the linear programming problem
result = linprog(c=cost, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method="simplex")

# Check if the optimization was successful
if result.success:
    print("Optimal solution found with a total cost of:", result.fun)
    print("Shipment quantities for each route:")
    for i, qty in enumerate(result.x):
        print(f"Route {i+1}: {qty} units")
else:
    print("Optimization failed:", result.message)


Optimal solution found with a total cost of: 285.0
Shipment quantities for each route:
Route 1: 15.0 units
Route 2: 0.0 units
Route 3: 5.0 units
Route 4: 0.0 units
Route 5: 25.0 units
Route 6: 5.0 units
Route 7: 0.0 units
Route 8: 0.0 units
Route 9: 25.0 units


<ipython-input-1-9d58628dfdd9>:36: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c=cost, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method="simplex")
<ipython-input-1-9d58628dfdd9>:36: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  result = linprog(c=cost, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method="simplex")


In [ ]:
!pip install pulp
# Import PuLP modeler functions
from pulp import *

# Creates a list of all the supply nodes
factories = ["A", "B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 50, "B": 350, "C": 100}

# Creates a list of all demand nodes
projects = ["1", "2", "3"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {
    "1": 50,
    "2": 150,
    "3": 300,
}

# Intermediate nodes
warehouses = ["P", "Q"]

# Creates a list of costs of each transportation path
costs_1 = [  # warehouses
    [2, 1],  # A factories
    [4, 5],  # B
    [1.5, 0.5]  # C
]

costs_2 = [  # projects
    [5, 2, 2],  # P warehouses
    [4, 1, 3],  # Q
]

# The cost data is made into a dictionary
costs_1 = makeDict([factories, warehouses], costs_1, 0)
costs_2 = makeDict([warehouses, projects], costs_2, 0)

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Material Supply Problem", LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes_1 = [(f, w) for f in factories for w in warehouses]
Routes_2 = [(w, p) for w in warehouses for p in projects]

# A dictionary called 'Vars_1' is created for routes from factories to warehouses
vars_1 = LpVariable.dicts("Route_1", (factories, warehouses), 0, None, LpInteger)

# A dictionary called 'Vars_2' is created for routes from warehouses to projects
vars_2 = LpVariable.dicts("Route_2", (warehouses, projects), 0, None, LpInteger)

# The objective function is added to 'prob' first
prob += (
    lpSum([vars_1[f][w] * costs_1[f][w] for (f, w) in Routes_1]) +
    lpSum([vars_2[w][p] * costs_2[w][p] for (w, p) in Routes_2]),
    "Sum_of_Transporting_Costs",
)

# The supply maximum constraints are added to prob for each supply node (factories)
for f in factories:
    prob += (
        lpSum([vars_1[f][w] for w in warehouses]) <= supply[f],
        "Sum_of_Products_out_of_factories_%s" % f,
    )

# The demand minimum constraints are added to prob for each demand node (project)
for p in projects:
    prob += (
        lpSum([vars_2[w][p] for w in warehouses]) >= demand[p],
        "Sum_of_Products_into_projects_%s" % p,
    )

# Transshipment constraints: What is shipped into a transshipment node must be shipped out.
for w in warehouses:
    prob += (
        lpSum([vars_1[f][w] for f in factories]) - lpSum([vars_2[w][p] for p in projects]) == 0,
        "Sum_of_Products_out_of_warehouse_%s" % w,
    )

# The problem is solved using PuLP's choice of Solver
prob.solve()

# Print the variables optimized value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimized objective function value is printed to the screen
print("Value of Objective Function = ", value(prob.objective))


Route_1_A_P = 0.0
Route_1_A_Q = 50.0
Route_1_B_P = 300.0
Route_1_B_Q = 50.0
Route_1_C_P = 0.0
Route_1_C_Q = 100.0
Route_2_P_1 = 0.0
Route_2_P_2 = 0.0
Route_2_P_3 = 300.0
Route_2_Q_1 = 50.0
Route_2_Q_2 = 150.0
Route_2_Q_3 = 0.0
Value of Objective Function =  2500.0


In [ ]:
!pip install scipy

from scipy.optimize import linear_sum_assignment
import numpy as np

# Define the cost matrix
cost_matrix = np.array([
    [13, 8, 10, 15],
    [9, 14, 12, 7],
    [11, 13, 9, 12],
    [12, 10, 14, 6]
])

# Solve the assignment problem
row_ind, col_ind = linear_sum_assignment(cost_matrix)

# Calculate the minimum cost
min_cost = cost_matrix[row_ind, col_ind].sum()

# Print the optimal assignment and the minimum cost
print("Optimal assignment:")
for worker, task in zip(row_ind, col_ind):
    print(f"Worker {worker + 1} is assigned to Task {chr(task + 65)}")

print(f"Total minimum cost (in hours): {min_cost}")


Optimal assignment:
Worker 1 is assigned to Task B
Worker 2 is assigned to Task A
Worker 3 is assigned to Task C
Worker 4 is assigned to Task D
Total minimum cost (in hours): 32
